<h1>Is this the perfect spot for opening a Pizzeria in Bucharest - Romania?</h1>

<h2>Introduction/Business Problem</h2>

<h4>A description of the problem and a discussion of the background.</h4>

<h4>This study aims to give extra infos to help people planning to open a Pizzeria in Romania's capital - Bucharest. 
In order to make an informed decision about the location i will try to provide data about the population of each of district as well as the competitors already present on the same district.</h4>

i will get the infos about the population desity from here:

https://en.wikipedia.org/wiki/Sectors_of_Bucharest

<h4>Know Your Neighbors
When looking for a restaurant location, consider who else is doing business in the neighborhood.
Are there already half a dozen Pizzeria in the neighborhood? </h4>

i will get infos about other pizzerias in Bucharest from Foursquare API and i will get place them in districts according to a cvs files that contains all the Postal codes in every district.

<h2>FIRST LETS SEE THE PIZZERIAS</h4>

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


the next cell will be deleted because it contains log in to Foursquare infos

In [4]:
xxxx

NameError: name 'xxxx' is not defined

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [ ]:
#seting the adress to bucharest and verifing by checking if the latitude and longitude are 44.4361414 26.1027202

address = 'BUCURESTI'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

In [ ]:
#searching for all the pizzerias in a 15 km radius(that will cover the hole Bucharest)
search_query = "Pizzeria"
radius = 15000
print(search_query + ' .... OK!')

#### Define the corresponding URL

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

#### Send the GET Request and examine the results

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(2)

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(2)

#### Let's visualize the Italian restaurants that are nearby on the map

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Bucharest

# add a red circle marker to represent the Bucharest
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Bucuresti',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<h2>Second lets see the district they are located</h2>

In [ ]:
# here we will clean the data so that we can see witch sector has the least Pizzerias
df1=dataframe_filtered
df1.head(2)

In [ ]:
df1.columns

In [ ]:
df1.drop(['categories', 'lat', 'lng', 'labeledLatLngs', 'distance', 'cc', 'city', 'state', 'country', 'formattedAddress','crossStreet', 'neighborhood', 'id'], axis=1, inplace=True)

In [ ]:
df1.head(3)

In [ ]:
df1.xs(0)['postalCode']="030119"
df1.xs(2)['postalCode']="010784"
df1.xs(5)['postalCode']="020616"
df1.xs(6)['postalCode']="021151"
df1.xs(7)['postalCode']="014453"
df1.xs(8)['postalCode']="021151"
df1.xs(9)['postalCode']="040342"
df1.xs(10)['postalCode']="024102"
df1.xs(11)['postalCode']="031623"
df1.xs(12)['postalCode']="041322"
df1.xs(13)['postalCode']="031281"
df1.xs(14)['postalCode']="041322"
df1.xs(15)['postalCode']="030441"
df1.xs(16)['postalCode']="012244"
df1.xs(17)['postalCode']="022627"
df1.xs(18)['postalCode']="021194"
df1.xs(19)['postalCode']="042092"
df1.xs(20)['postalCode']="077190"
df1.xs(21)['postalCode']="030167"
df1.xs(22)['postalCode']="050455"
df1.xs(23)['postalCode']="013813"
df1.xs(24)['postalCode']="030167"
df1.xs(25)['postalCode']="062204"
df1.xs(26)['postalCode']="040241"
df1.xs(27)['postalCode']="077190"
df1.xs(28)['postalCode']="010934"
df1.xs(29)['postalCode']="012329"

In [ ]:
df1.head(30)

In [ ]:
df1.drop(['address'], axis=1, inplace=True)

In [ ]:
df1.rename(columns={'postalCode':'Codpostal'}, inplace=True)
df1

<h3>Import a csv file with all the postal codes and districts</h3>

In [ ]:
df2=pd.read_csv("cod postal buc.csv", sep='; ', delimiter=";", header='infer', names=None, index_col=None, error_bad_lines=False)
df2.columns
df2

In [ ]:
df2.drop(['Tip artera', 'Denumire artera', 'Numar','Oficiu distribuire', 'SIRUTA SECTOR', 'NIV', 'SIRSUP'], axis=1, inplace=True)

In [ ]:
#making the data intoathe same format - object
df2.Codpostal = df2.Codpostal.astype(str)
df2.Sector = df2.Sector.astype(str)
df1.Codpostal = df1.Codpostal.astype(str)
df1.Codpostal = df1.Codpostal.astype(str)

In [ ]:
#inner join between the 2 data base
df3 = pd.merge(left=df1, right=df2, how='left', left_on='Codpostal', right_on='Codpostal')
df3


In [ ]:
#remove NaN
df3=df3.dropna()

In [ ]:
#group the data to see how manny Pizzerias are per District
df3=df3.groupby(['Sector'])['name'].count().reset_index()
df3.rename(columns={'name':'Number of pizzeria', 'Sector':'District'}, inplace=True)
df3

<h2>Third - lets see the the population densitity of every district of Bucharest</h2>

<h3>we will scrape a wiki page with the relevant table</h3>


In [ ]:
#import libraries
import urllib.request
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [ ]:
url = "https://en.wikipedia.org/wiki/Sectors_of_Bucharest"

In [ ]:
page = urllib.request.urlopen(url)

In [ ]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [ ]:
#The fourth table is relevant for my study, the one with the density
right_table = soup.find_all('table')[3] # Grab the first table
right_table

In [ ]:
# Loop through the rows
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [ ]:
#make a dataframe out of the wiki table
df4=pd.DataFrame(A,columns=['Rank'])
df4['District']=B
df4['Population density']=C
df4

Replacing <em> /n </em> with empy space and the ',' from the numbers

In [ ]:
df4 = df4.replace('\n','', regex=True)
df4 = df4.replace(',','', regex=True)
df4 = df4.replace('Sector ','', regex=True)
df4.drop(["Rank"], axis=1, inplace=True)
df4

<h3>Inner join between the 2 data bases in order to see how manny Pizzerias in every discrict and what is the population density</h3>

In [ ]:
df5 = pd.merge(left=df3, right=df4, how='left', left_on='District', right_on='District')

<h3>Data normalization and substractig the normalize data in order to make top of the best Pizzeria districts. The ideal district is one with the lowest number of pizzerias and the highest density of the population</h3>

In [ ]:
import pandas as pd
from sklearn import preprocessing

x = df5.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df5 = pd.DataFrame(x_scaled)

In [ ]:
#making a column of the Districts 
df5['District'] = df5.index+1
#rename the columns
df5.rename(columns={1:'Number_of_pizzerias', 2:'Population_density'}, inplace=True)
#drop the first column
df5.drop(0, axis=1, inplace=True)


In [ ]:
df5

In [ ]:
#creating a column "Score_diff" that represents the rank of the districts
df5.Number_of_pizzerias = df5.Number_of_pizzerias.astype(object)
df5.Population_density = df5.Population_density.astype(object)
#making the type of Score_diff float and substractig Number_of_pizzerias from Population_density
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df5['Score_diff_RANK']=df5['Population_density'] - df5['Number_of_pizzerias']
df5.Score_diff_RANK = df5.Score_diff_RANK.astype(float)
df5

In [ ]:
final_df = df5.sort_values(by=['Score_diff_RANK'], ascending=False)

In [ ]:
final_df

<h2>The worst district to open a pizzeria in Bucharest is <ins>DISTRICT 1</ins></h2>

<h3>It has the lowest density of population and the second number of pizzerias per district</h3>

<h2>The best district to open a pizzeria in Bucharest is <ins>DISTRICT 5</ins></h2>

<h3>it has the lowest number of pizzerias and the 4th populations density per district </h3>